<a href="https://colab.research.google.com/github/Minty-cyber/ShadowFox-AI-ML/blob/main/Loan_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the necessary libraries needed for development.

In [95]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder



In [64]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/loan_prediction.csv')

df.head()

print(df.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(614, 13)


Dropping missing values in your dataset

In [65]:
df_dropped= df.dropna()
print(df_dropped.shape)

(480, 13)


Checking the number of columns present after

In [99]:
X = df_dropped.drop(['Loan_ID', 'Loan_Status'], axis=1)
y = df_dropped['Loan_Status']

In [100]:
numeric_cols = X.select_dtypes(include=np.number).columns
categorical_cols = list(set(X.columns) - set(numeric_cols))

# Define transformers for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ])

In [101]:
X_processed = preprocessor.fit_transform(X)

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [106]:
# Convert y_train and y_test to numeric type before converting to tensors
y_train_numeric = y_train.map({'Y': 1, 'N': 0}).astype(float)
y_test_numeric = y_test.map({'Y': 1, 'N': 0}).astype(float)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_numeric.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test_numeric.values, dtype=torch.float32).view(-1, 1)

In [ ]:
print(f"X_train shape: {X_train_tensor.shape}")
print(f"X_test shape: {X_test_tensor.shape}")
print(f"y_train shape: {y_train_tensor.shape}")
print(f"y_test shape: {y_test_tensor.shape}")

In [112]:
class LoanApprovalModel(nn.Module):
    def __init__(self, input_size):
        super(LoanApprovalModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # First fully connected layer, input_size to 64
        self.fc2 = nn.Linear(64, 32)          # Second fully connected layer, 64 to 32
        self.fc3 = nn.Linear(32, 1)           # Third fully connected layer, 32 to 1 (output)
        self.relu = nn.ReLU()                 # ReLU activation function
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
          x = self.relu(self.fc1(x))  # Pass through first layer and apply ReLU activation
          x = self.relu(self.fc2(x))  # Pass through second layer and apply ReLU activation
          x = self.sigmoid(self.fc3(x))  # Pass through third layer and apply Sigmoid activation for binary output
          return x

In [113]:
input_size = X_train_tensor.shape[1]

# Instantiate the model
model = LoanApprovalModel(input_size)

In [ ]:


# Define the loss function and the optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Number of epochs
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Clear the gradients
    outputs = model(X_train_tensor)  # Forward pass
    loss = criterion(outputs, y_train_tensor)  # Compute loss
    loss.backward()  # Backward pass (compute gradients)
    optimizer.step()  # Update weights

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [117]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # No need to compute gradients for evaluation
    outputs = model(X_test_tensor)
    predicted = (outputs > 0.5).float()  # Convert probabilities to binary (0 or 1)
    accuracy = (predicted == y_test_tensor).float().mean()  # Compute accuracy

print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.8125
